# Motion across sessions - comparisson with the cemented dataset

These will go in sup, perhaps all combined would be nice.

In [ ]:
from labdata.schema import *
import pylab as plt
%matplotlib widget
from labdata import chronic_paper as paper
plt.matplotlib.rcParams['pdf.fonttype'] = 42
plt.matplotlib.rcParams['ps.fonttype'] = 42
savepath = Path('../../figures/figure3')
savepath.mkdir(parents = True,exist_ok = True)


In [ ]:
%matplotlib widget
key = dict(subject_name="_AL032",shank_num=0)
import pylab as plt
plt.figure()
paper.ConcatenatedSpikes().plot_raster(key, corrected=False, overlay_dredge=True,cmap = 'gray')
plt.axis('tight')
# plt.ylim(4200, 5200)

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "_AL031"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0]))
intervals_days_cemented = intervals_days.astype('timedelta64[D]') # these are the intervals for which we have _AL036 data


# selected JC131 sessions
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "JC131"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')
arg_sessions = []
for i in intervals_days_cemented:
    arg_sessions.append(np.where(intervals_days<=i)[0][-1])
arg_sessions
intervals_days,intervals_days_cemented,arg_sessions

In [ ]:
data_AL036 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "_AL036"' & 'shank_num = 0').fetch1()
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "_AL036"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0]))
intervals_days_cemented = intervals_days.astype('timedelta64[D]') # these are the intervals for which we have _AL036 data

data_AL036['spike_times_s'] = data_AL036['spike_times_s'].astype(np.float32)
displ = data_AL036['displacement']
sp= data_AL036['spike_times_s']

from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(sp,data_AL036['spike_depths_um'],data_AL036['spike_amps'],cmap = 'gray_r',clim = [0,300])

offsets = np.hstack([data_AL036['session_breaks'],np.max(sp)])

plt.vlines(offsets,3500,4500,'k',lw = 0.5)
plt.ylim([3500,4500])
plt.plot(displ+np.mean([3500,4500]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days_cemented.astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('AL036 cortex (shank 0)')
plt.savefig(savepath/f'AL036_shank0_drift.pdf')

In [ ]:
shank = 3
data_AL032 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "_AL032"' & f'shank_num = {shank}').fetch1()
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "_AL032"'& f'shank_num = {shank}').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0]))
intervals_days_cemented = intervals_days.astype('timedelta64[D]') # these are the intervals for which we have _AL036 data

data_AL032['spike_times_s'] = data_AL032['spike_times_s'].astype(np.float32)
displ = data_AL032['displacement']
sp= data_AL032['spike_times_s']

from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(sp,data_AL032['spike_depths_um'],data_AL032['spike_amps'],cmap = 'gray_r',clim = [0,300])

plt.vlines(offsets,2700,3700,'k',lw = 0.5)
plt.ylim([2700,3700])
plt.plot(displ+np.mean([2700,3700]),'r')
offsets = np.hstack([data_AL032['session_breaks'],np.max(sp)])
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days_cemented.astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title(f'AL032 cortex (shank {shank})')
plt.savefig(savepath/f'AL032_shank{shank}_drift.pdf')

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "JC131"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')

idx_JC131 = [0,1,8,30,34,46,59,68,76,77,78,84,87]

data_JC131 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "JC131"' & 'shank_num = 0').fetch1()

data_JC131['spike_times_s'] = data_JC131['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([4,data_JC131['session_breaks']]),
                          np.hstack([data_JC131['session_breaks'],np.max(data_JC131['spike_times_s'])])]).T[idx_JC131]
t = data_JC131['time_bin_centers_s']
displ = data_JC131['displacement'][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_JC131['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_JC131['spike_depths_um'][idx],data_JC131['spike_amps'][idx],cmap = 'gray_r',clim = [0,300])

plt.vlines(offsets,4200,5200,'k',lw = 0.5)
plt.ylim([4200,5200])
plt.plot(displ+np.mean([4200,5200]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_JC131].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('JC131 cortex (shank 0)')
plt.savefig(savepath/f'JC131_shank0_drift.pdf')

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "JC131"'& 'shank_num = 2').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')

idx_JC131 = [0,1,8,30,34,46,59,68,76,77,78,84,87]

data_JC131 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "JC131"' & 'shank_num = 2').fetch1()

data_JC131['spike_times_s'] = data_JC131['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([0,data_JC131['session_breaks']]),
                          np.hstack([data_JC131['session_breaks'],np.max(data_JC131['spike_times_s'])])]).T[idx_JC131]
t = data_JC131['time_bin_centers_s']
displ = data_JC131['displacement'][2][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_JC131['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_JC131['spike_depths_um'][idx],data_JC131['spike_amps'][idx],cmap = 'gray_r',clim = [0,300])

plt.vlines(offsets,1300,2300,'k',lw = 0.5)
plt.ylim([1300,2300])
plt.plot(displ+np.mean([1300,2300]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_JC131].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('JC131 thalamus (shank 2)')
plt.savefig(savepath/f'JC131_shank2_drift.pdf')

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "MM012"'&
                  'probe_num = 2' & 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')
idx_MM012 = [0,1,6,18, 19, 20,24,32, 33, 40, 41 ,44,48]
data_MM012 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "MM012"' 
              & 'probe_num = 2' 
              & 'shank_num = 0').fetch1()


data_MM012['spike_times_s'] = data_MM012['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([4,data_MM012['session_breaks']]),
                          np.hstack([data_MM012['session_breaks'],np.max(data_MM012['spike_times_s'])])]).T[idx_MM012]
t = data_MM012['time_bin_centers_s']
displ = data_MM012['displacement'][2][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_MM012['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_MM012['spike_depths_um'][idx],data_MM012['spike_amps'][idx],cmap = 'gray_r',clim = [0,70])

plt.vlines(offsets,2800,3800,'k',lw = 0.5)
plt.ylim([2800,3800])
plt.plot(displ+np.mean([2800,3800]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_MM012].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('MM012 cortex (NP1)')
plt.savefig(savepath/f'MM012_shank2_drift.pdf')

In [ ]:

intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "MM012"'&
                  'probe_num = 1' & 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')
idx_MM012 = [0,1,6,18, 19, 20,24,32, 33, 40, 41 ,44,48]
data_MM012 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "MM012"' 
              & 'probe_num = 1' 
              & 'shank_num = 0').fetch1()


data_MM012['spike_times_s'] = data_MM012['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([4,data_MM012['session_breaks']]),
                          np.hstack([data_MM012['session_breaks'],np.max(data_MM012['spike_times_s'])])]).T[idx_MM012]
t = data_MM012['time_bin_centers_s']
displ = data_MM012['displacement'][-2][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_MM012['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_MM012['spike_depths_um'][idx],data_MM012['spike_amps'][idx],cmap = 'gray_r',clim = [0,70])

plt.vlines(offsets,2600,3600,'k',lw = 0.5)
plt.ylim([2600,3600])
plt.plot(displ+np.mean([2600,3600]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_MM012].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('MM012 motor cortex (NP1)')
plt.savefig(savepath/f'MM012_shank0_drift.pdf')

# Example of session to session motion estimate.

In [ ]:
from labdata.schema import *
import pylab as plt
%matplotlib widget
from labdata import chronic_paper as paper
plt.matplotlib.rcParams['pdf.fonttype'] = 42
plt.matplotlib.rcParams['ps.fonttype'] = 42
savepath = Path('../../figures/figure3')
savepath.mkdir(parents = True,exist_ok = True)


In [ ]:
intervals_days

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedSessions()*Session() & 'subject_name = "JC131"'& 'shank_num = 2').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')

idx_JC131 = [1,9,18,24,28,35,41,51,53,59,64,65,69,76,79,80,90,97,99,103,106,111,114,119,123,128]

#data_JC131 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "JC131"' & 'shank_num = 2').fetch1()

data_JC131['spike_times_s'] = data_JC131['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([0,data_JC131['session_breaks']]),
                          np.hstack([data_JC131['session_breaks'],np.max(data_JC131['spike_times_s'])])]).T[idx_JC131]
t = data_JC131['time_bin_centers_s']
displ = data_JC131['displacement'][0][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_JC131['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [8,1.5])
plot_drift_raster(np.hstack(nsp),
                  data_JC131['spike_depths_um'][idx],
                  data_JC131['spike_amps'][idx],
                  cmap = 'gray_r',clim = [0,300],
                  markersize = 0.001,
                  n_spikes_to_plot = 150000)

plt.vlines(offsets,1300,2300,'k',lw = 0.5)
plt.ylim([1400,2150])
plt.plot(displ+np.mean([1300,2300]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_JC131].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('JC131 thalamus (shank 2)')
plt.savefig(savepath/f'longitudinal_JC131_shank2_motion.pdf')

In [ ]:
paper.DredgeSpikeDetection() & 'subject_name = "MM008"'

In [ ]:
subjects = ['JC131', 'MM012', 'MM013', '_AL031', '_AL032','_AL036']
subjects = ['JC131', 'MM018','MM013','_AL031', '_AL032','_AL036']
def compute_intersession_drift(displacement, session_breaks):
    lims = np.concatenate([np.array([0]), session_breaks])
    avg_pos = []
    for start, end in zip(lims[:-1], lims[1:]):
        avg_pos.append(np.mean(displacement[start:end])) # mean position per session
    diffs = np.diff(avg_pos)
    return avg_pos, diffs
    
# subjects = np.unique(paper.ConcatenatedSpikes.fetch('subject_name'))
violinres = []
caption = []
for subject in subjects:
    res = pd.DataFrame(((paper.ConcatenatedSpikes.DredgeResults() * paper.ConcatenatedSpikes().proj('session_breaks') 
                         & f"subject_name = '{subject}'")).fetch())
    for i,r in res.iterrows():
        displacement = r['displacement']
        if len(displacement.shape)==1:
            displacement = [displacement]
        avg_pos = []
        diffs = []
        if 'MM018' in subject and r.shank_num and i>=1:
            continue
        for i,d in enumerate(displacement):
            a,d = compute_intersession_drift(d,r['session_breaks'])

            if not np.mean(np.abs(a)) < 2:
                avg_pos.append(a)
                diffs.append(a)
        if len(diffs):
            
            violinres.append(dict(subject_name = subject, 
                                  probe_num = r.probe_num,
                                  shank_num = r.shank_num,
                                  pos_average = avg_pos,
                                  dates = (paper.ConcatenatedSpikes.IncludedSessions()*Session() 
                                           & f'subject_name = "{subject}"').fetch('session_datetime'),
                                 pos_difference = diffs))
            
            caption.append(subject + f' probe {r.probe_num} shank {r.shank_num}')
violinres = pd.DataFrame(violinres)

# res['spatial_bin_centers_um'].iloc[0]
# violinres

In [ ]:
fig = plt.figure(figsize = [8,3])
fig.add_axes([0.2,0.4,0.7,0.5])
dd = violinres.pos_difference.values
values = [np.abs(np.stack(d).flatten()) for d in dd]
ii = [np.random.uniform(0,0.1,size = d.shape)+i for i,d in enumerate(values)]

viol = plt.violinplot(values,showmedians=True);
plt.xticks(1+np.arange(len(violinres)),[f'{r.subject_name} shank {r.shank_num}' for i,r in violinres.iterrows()],rotation = 90,fontsize = 6);
for i,b in enumerate(viol['bodies']):
    c = 'k'
    b.set_edgecolor(c)
    if 'AL' in caption[i]:
        b.set_facecolor('gray')
    else:
        b.set_facecolor('lightgray')
    b.set_alpha(1)
viol['cmedians'].set_color('k')
viol['cbars'].set_color('k')
viol['cmins'].set_color('k')
viol['cmaxes'].set_color('k')
plt.xticks(1+np.arange(len(violinres)),caption)
plt.gca().spines[['top','right']].set_visible(False)
plt.ylim([0,200])
plt.ylabel('delta session motion', fontsize = 12)
plt.savefig(savepath/f'all_probe_motion_violins_MM018.pdf')

In [ ]:
# res = pd.DataFrame(paper.ConcatenatedSpikes.DredgeResults() & key).fetch())
# for s in subjects:
# (paper.ConcatenatedSpikes() & key)


# subject_sessions = (paper.ConcatenatedSpikes.IncludedSessions()*Session()).fetch('session_datetime')

In [ ]:


#intersession_positions, intersession_drifts = [(compute_intersession_drift(m, lims)) for m in shank_motion_estimates]
# intersession_drifts = [compute_intersession_drift(m, lims) for m in shank_motion_estimates]

np.nanmin(anova_table['PR(>F)'])

In [ ]:
# stats for violin plot
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
 
dat = violinres[['subject_name','shank_num','pos_difference']].copy()
dat = dat.explode('pos_difference').explode('pos_difference').reindex()
dat['pos_difference'] = dat['pos_difference'].astype(float)
#model = ols('pos_difference ~ C(subject_name) + C(shank_num) + C(subject_name):C(shank_num)', data=dat).fit()
model = ols('pos_difference ~ C(subject_name)*C(shank_num)', data=dat).fit()
#print(model.summary())
anova_table = sm.stats.anova_lm(model, typ=3)
print(anova_table)
tukey_animal = pairwise_tukeyhsd(endog=dat['pos_difference'], groups=dat['subject_name'], alpha=0.05)
print(tukey_animal)
tukey_shank = pairwise_tukeyhsd(endog=dat['pos_difference'], groups=dat['shank_num'].values, alpha=0.05)
print(tukey_shank)

### Comparisson of unit yield with cemented probes 

Chosen only sessions with probes in V1

In [ ]:
SAVEPATH = Path('../../figures/figure3')
SUBJECTS = ['_AL031','_AL032','_AL036','MM013','MM018','JC131']
PROBE_NUMS = [0,0,0,2,2,0]

cemented_subjects = SUBJECTS[:3]
chronic_holder_subjects = SUBJECTS[3:]

subjects = '(' + ','.join([f'"{s}"' for s in SUBJECTS]) + ')'
dat = pd.DataFrame()
for s,p in zip(SUBJECTS,PROBE_NUMS):
    unit_counts = Session() * UnitCount() & dict(subject_name=s,
                                                 probe_num=p,
                                                 parameter_set_num=5,
                                                 unit_criteria_id=1)
    subject_data = unit_counts.proj('session_datetime','all','sua','mua').fetch(format='frame')
    dat = pd.concat([dat,subject_data])
dat = dat.reset_index()

cemented_sessions = dat[dat.subject_name.isin(cemented_subjects)]
chronic_holder_sessions = dat[dat.subject_name.isin(chronic_holder_subjects)]

# randomly subsample our chronic holder mice so that they're equal in number for randomly selecting sessions
min_num_sessions = chronic_holder_sessions.subject_name.value_counts().min()
subsample_group = lambda group, n_per_subject:  group.sample(n=n_per_subject, replace=True) 
chronic_holder_sessions = chronic_holder_sessions.groupby('subject_name', group_keys=False).apply(subsample_group, n_per_subject=min_num_sessions)
print(f'The minimum number of sessions in both groups is: {min_num_sessions}')

In [ ]:
# choose randomized sets but an equal number of days apart
n_random = 3 
n_to_pick = 1
rnd = np.random.default_rng(seed = 23)
c_mice,h_mice,h_deltadays, c_deltadays, c_sua, c_mua, h_mua, h_sua = [],[],[],[],[],[],[],[]
for i,cemented_session in cemented_sessions.iterrows():
    cem_delta_days = (cemented_session.session_datetime - cemented_sessions.session_datetime.min()).days
    hol_delta_days = np.array([x.days for x in chronic_holder_sessions.session_datetime - chronic_holder_sessions.session_datetime.min()])
    sorted_day_distance = np.argsort(np.abs(hol_delta_days-cem_delta_days))
    ff = rnd.choice(sorted_day_distance[:n_random],n_to_pick,replace=False)
    comparison_sessions = chronic_holder_sessions.iloc[ff]
    #print(cemented_session.subject_name, cemented_session.session_name, comparison_sessions.iloc[0].subject_name, comparison_sessions.iloc[0].session_name)
    c_sua.append(cemented_session.sua)
    c_mua.append(cemented_session.mua)
    deltadays = cemented_session.session_datetime - cemented_sessions.session_datetime.min()
    c_deltadays.append(deltadays.days)
    c_mice.append(cemented_session.subject_name)

    h_sua.append(comparison_sessions.sua.mean())
    h_mua.append(comparison_sessions.mua.mean())
    deltadays = comparison_sessions.session_datetime - chronic_holder_sessions.session_datetime.min()
    h_deltadays.append(deltadays.iloc[0].days)
    h_mice.append(comparison_sessions.iloc[0].subject_name)
# plot the number of units for several session from the cemented probes and the chronic holder

_, h_cols = np.unique(h_mice, return_inverse=True)
_, c_cols = np.unique(c_mice, return_inverse=True)

labels = ['Cemented','Chronic holder'] * 2
leg_labels = []
xticks = (1,2,3.5,4.5)
scatter_pos = []
fig = plt.figure(figsize = [3,3])

for i,(pos,n_units) in enumerate(zip(xticks,[c_mua, h_mua, c_sua, h_sua])):
    scatter_positions = rnd.normal(0, scale=.05, size=len(n_units))
    scatter_pos.append(scatter_positions+pos)
    if i % 2 == 0:
        plt.scatter(pos+scatter_positions, n_units, s=25, c=c_deltadays, alpha=.7, cmap='plasma',edgecolors = 'none')
    else:
        plt.scatter(pos+scatter_positions, n_units, s=25, c=h_deltadays, alpha=.7, cmap='plasma',edgecolors = 'none')

    plt.hlines(np.median(n_units), pos-.3, pos+.3, color='black')

nsess = len(c_mua)
plt.plot(np.stack(scatter_pos[:2]), np.stack([c_mua, h_mua], axis=1).T, 
         linewidth=.5, color='black', alpha=.5)
plt.plot(np.stack(scatter_pos[2:]), np.stack([c_sua, h_sua], axis=1).T, 
         linewidth=.5, color='black', alpha=.5)
# plt.plot(np.repeat(np.expand_dims(xticks[2:],1),nsess, axis=1), np.stack([c_sua, h_sua], axis=1).T, 
#          linewidth=.5, color='black', alpha=.5)

plt.colorbar(label='days from first recording',shrink = 0.2)

plt.xticks(xticks, labels)
plt.xticks(rotation=90)
plt.yticks([0,300,600])
plt.ylabel('Unit yield')
plt.xlabel('')
plt.gca().spines[['right', 'top']].set_visible(False)
plt.savefig(SAVEPATH / 'cemented_comparison_unit_yield.pdf', bbox_inches='tight', dpi=300, format='pdf')


In [ ]:
from scipy.stats import ks_2samp
ks_2samp(c_sua,h_sua),np.mean(c_sua),np.std(c_sua)/np.sqrt(len(h_sua)),np.mean(h_sua),np.std(h_sua)/np.sqrt(len(h_sua))
# ks_2samp(c_mua,h_mua),np.median(c_mua),np.median(h_mua)

In [ ]:
# units yield over time
# plt.figure(figsize = (3,3))
plt.figure(figsize = (10,2))
markers = ['.','s','^','d','<','D','>','p','>','h','H','.','s','^','d','|','_']
c = 0
# subjects = ['JC131', 'MM008', '_AL031', '_AL036']
subjects = ['JC131', 'JC162','JC142','MM018','MM013','_AL031', '_AL036','_AL032']
probes = [0,2,2,0,0,0]
parameter_set_num = 5
unit_criteria_id = 1
color = '#d62728'
for i,s in enumerate(subjects):
    probenums = (EphysRecording.ProbeSetting & dict(subject_name=s, parameter_set_num=5)).fetch('probe_num')
    n_probes = np.unique(probenums)
    for p in n_probes:
        config_ids = (Session * EphysRecording.ProbeSetting * UnitCount & dict(subject_name=s, probe_num=p, 
                                                                               parameter_set_num=parameter_set_num,
                                                                               unit_criteria_id=unit_criteria_id)).fetch('configuration_id')
        ids, counts = np.unique(config_ids, return_counts=True)
        config_id = ids[np.argmax(counts)]
        print(f'Using configuration ID {config_id} for subject {s}, probe {p}')
        query = EphysRecording.ProbeSetting * Session * UnitCount & dict(probe_num=p, subject_name=s, 
                                                                         configuration_id=config_id, 
                                                                         unit_criteria_id=unit_criteria_id,
                                                                         parameter_set_num=parameter_set_num)
        # find the indices for the closest sessions to the target days
        datetimes, nsua = query.fetch('session_datetime','sua')
        delta_days = (datetimes - datetimes[0]).astype('timedelta64[D]').astype(int)
        target_delta = 10
        targeted_days = np.arange(np.random.uniform(0,target_delta/2), np.max(delta_days)+target_delta+1, target_delta).astype(int)
        session_inds_to_use = np.unique([np.absolute(delta_days-target_day).argmin() for target_day in targeted_days])

        label = f'{s.replace("_","")} probe {p}'
        #plt.plot(delta_days[session_inds_to_use], nsua[session_inds_to_use], color='black', alpha=0.5, linewidth=1)
        if '_' in s:
            plt.scatter(delta_days[session_inds_to_use], nsua[session_inds_to_use], s=10, color='black',facecolors = 'none', marker=markers[c], label=label,alpha = 1)
            plt.plot(delta_days[session_inds_to_use], nsua[session_inds_to_use], color='black', alpha=0.7, linewidth=1)
        else:
            plt.scatter(delta_days[session_inds_to_use], nsua[session_inds_to_use], s=10,  color=color, facecolors = 'none', marker=markers[c], label=label,alpha = 1)
            plt.plot(delta_days[session_inds_to_use], nsua[session_inds_to_use], color=color, alpha=0.7, linewidth=1)
            
        c+=1
    
plt.ylim(0,400)
plt.gca().spines[['right', 'top']].set_visible(False)
plt.xlabel('Days from first recording')
plt.ylabel('N single units')
# plt.legend()
plt.yticks([0,150,300])
plt.xticks([0,60,120,180,240,300])
# plt.savefig(savepath/f'subsampled_unit_counts_over_time.pdf')
plt.ylabel('# simultaneously\n recorded single units\n(log)');
plt.savefig(savepath/f'subsampled_unit_counts_over_time_stretched.pdf')

plt.ylim([10,1000])
plt.gca().set_yscale('log')
plt.yticks([5,50,500],[5,50,500])
plt.ylabel('# simultaneously\n recorded single units\n(log)');
plt.savefig(savepath/f'subsampled_unit_counts_over_time_stretched_log.pdf')
